In [63]:
import sqlalchemy as sqla 
from sqlalchemy import create_engine 
import traceback 
import glob 
import os 

import requests 
import time 
import json


from IPython.display import display


import pymysql

In [64]:
import requests
import traceback
import datetime
import time
import os

In [65]:
URI="dbbikes.cjk4ybuxtkwv.us-east-1.rds.amazonaws.com"
PORT="3306"
DB="dbbikes"
USER="cmcelduff"
PASSWORD="Tullamore1!"

In [66]:
#Connect to database
engine = create_engine("mysql+pymysql://{0}:{1}@{2}:{3}".format(USER, PASSWORD, URI, PORT), echo=True) 
connection = engine.connect()

2023-02-22 23:34:57,284 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-02-22 23:34:57,287 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-22 23:34:57,415 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-02-22 23:34:57,416 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-22 23:34:57,664 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-02-22 23:34:57,667 INFO sqlalchemy.engine.Engine [raw sql] {}


In [67]:
sql = """
CREATE DATABASE IF NOT EXISTS dublin_bikes;
"""
engine.execute(sql)

2023-02-22 23:35:00,706 INFO sqlalchemy.engine.Engine 
CREATE DATABASE IF NOT EXISTS dublin_bikes;

2023-02-22 23:35:00,707 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-22 23:35:00,831 INFO sqlalchemy.engine.Engine COMMIT


In [68]:
# command to check database is running
for res in engine.execute("SHOW VARIABLES;"):
    print(res)

2023-02-22 23:35:03,413 INFO sqlalchemy.engine.Engine SHOW VARIABLES;
2023-02-22 23:35:03,416 INFO sqlalchemy.engine.Engine [raw sql] {}
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1.2,TLSv1.3')
('authentication_policy', '*')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_offset', '1')
('autocommit', 'OFF')
('automatic_sp_privileges', 'ON')
('avoid_temporal_upgrade', 'OFF')
('awsauthenticationplugin_max_backoff_delay', '2000')
('awsauthenticationplugin_max_retry_count', '3')
('awsauthenticationplugin_retry_delay', '250')
('back_log', '61')
('basedir', '/rdsdbbin/mysql-8.0.28.R4/')
('big_tables', 'OFF')
('bind_address', '*')
('binlog_cache_size', '32768')
('binlog_checksum', 'CRC32')
('binlog_di

In [69]:
sql_create_schema = "CREATE SCHEMA `dublin_bikes`;"
try:
    #res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql_create_schema)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-22 23:35:06,723 INFO sqlalchemy.engine.Engine CREATE SCHEMA `dublin_bikes`;
2023-02-22 23:35:06,725 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-22 23:35:06,873 INFO sqlalchemy.engine.Engine ROLLBACK
(pymysql.err.ProgrammingError) (1007, "Can't create database 'dublin_bikes'; database exists")
[SQL: CREATE SCHEMA `dublin_bikes`;]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [70]:
# command to create table in mysql
sql_create_table = """
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`station` (
address VARCHAR(256),
banking INTEGER,
bike_stands INTEGER,
bonus INTEGER,
contact_name VARCHAR(256),
name VARCHAR(256),
number INTEGER,
position_lat REAL,
position_lng REAL,
status VARCHAR(256)
)
"""

try:
    #res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)



2023-02-22 23:35:09,560 INFO sqlalchemy.engine.Engine 
CREATE DATABASE IF NOT EXISTS dublin_bikes;

2023-02-22 23:35:09,563 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-22 23:35:09,696 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [71]:
sql_create_table = """
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`availability` (
number INTEGER,
available_bikes INTEGER,
available_bike_stands INTEGER,
last_update INTEGER
)
"""

try:
    #res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql_create_table)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-22 23:35:12,830 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`availability` (
number INTEGER,
available_bikes INTEGER,
available_bike_stands INTEGER,
last_update INTEGER
)

2023-02-22 23:35:12,832 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-22 23:35:13,054 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [72]:
NAME="Dublin"
STATIONS="https://api.jcdecaux.com/vls/v1/stations/"
DubBike_API = "7f06972a5ed335cf697379627fd13027274927c7"

def write_to_file(text):
    date_underscores = "{:%Y_%m_%d_%H_%M_%S}".format(datetime.datetime.now())
    with open(r"data\bikes_{}".format(date_underscores).replace(" ", "_"), "w") as f:
        f.write(text)

def stations_to_db(text):
    stations = json.loads(text)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (station.get("address"),int(station.get("banking")), int(station.get("bike_stands")), int(station.get("bonus")),station.get("contract_name"), station.get("name"), station.get("number"), station.get("position").get("lat"), station.get("position").get("lng"), station.get("status"))
        engine.execute("INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", vals)
    return

def availability_to_db(text):
    stations = json.loads(text)
    #print(availability)
    for station in stations:
        #print(station.get("name"))
        #print(station.get("number"))
        #print(station.get("available_bikes"))
        #print(station.get("available_bike_stands"))
        #print(station.get("last_update"))
        vals = (int(station.get("number")), int(station.get("available_bikes")), int(station.get("available_bike_stands")), int(station.get("last_update")))
        engine.execute("INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)", vals)
    return
        
                                            
def main():
    print(os.path)
    while True:
        try:
            now = datetime.datetime.now()
            r = requests.get(STATIONS, params={"apiKey": DubBike_API, "contract" : NAME})
            print(r, now)
            write_to_file(r.text)
            stations_to_db(r.text)
            availability_to_db(r.text)
            time.sleep(5*60)
        except:
            print(traceback.format_exc())
            
        return

if __name__== "__main__":
    main()

<module 'posixpath' from '/Users/conormcelduff/opt/anaconda3/envs/comp30830/lib/python3.10/posixpath.py'>
<Response [200]> 2023-02-22 23:35:15.720310
<class 'list'> 114
{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 9, 'available_bikes': 21, 'status': 'OPEN', 'last_update': 1677108390000}
2023-02-22 23:35:15,995 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 23:35:15,995 INFO sqlalchemy.engine.Engine [raw sql] ('Smithfield North', 0, 30, 0, 'dublin', 'SMITHFIELD NORTH', 42, 53.349562, -6.278198, 'OPEN')
2023-02-22 23:35:16,120 INFO sqlalchemy.engine.Engine COMMIT
{'number': 30, 'contract_name': 'dublin', 'name': 'PARNELL SQUARE NORTH', 'address': 'Parnell Square North', 'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526}, 'bank

2023-02-22 23:35:20,542 INFO sqlalchemy.engine.Engine [raw sql] ('Portobello Road', 0, 30, 0, 'dublin', 'PORTOBELLO ROAD', 43, 53.330091, -6.268044, 'OPEN')
2023-02-22 23:35:20,668 INFO sqlalchemy.engine.Engine COMMIT
{'number': 31, 'contract_name': 'dublin', 'name': 'PARNELL STREET', 'address': 'Parnell Street', 'position': {'lat': 53.350929, 'lng': -6.265125}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 9, 'available_bikes': 11, 'status': 'OPEN', 'last_update': 1677108409000}
2023-02-22 23:35:20,919 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 23:35:20,921 INFO sqlalchemy.engine.Engine [raw sql] ('Parnell Street', 0, 20, 0, 'dublin', 'PARNELL STREET', 31, 53.350929, -6.265125, 'OPEN')
2023-02-22 23:35:21,046 INFO sqlalchemy.engine.Engine COMMIT
{'number': 98, 'contract_name': 'dublin', 'name': 'FREDERICK STREET SOUTH', 'address': 'Frederick Street South', 'position': {'lat': 53.34

2023-02-22 23:35:25,501 INFO sqlalchemy.engine.Engine [raw sql] ('Earlsfort Terrace', 0, 30, 0, 'dublin', 'EARLSFORT TERRACE', 11, 53.334295, -6.258503, 'OPEN')
2023-02-22 23:35:25,622 INFO sqlalchemy.engine.Engine COMMIT
{'number': 17, 'contract_name': 'dublin', 'name': 'GOLDEN LANE', 'address': 'Golden Lane', 'position': {'lat': 53.340803, 'lng': -6.267732}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 20, 'available_bikes': 0, 'status': 'OPEN', 'last_update': 1677108506000}
2023-02-22 23:35:25,872 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 23:35:25,874 INFO sqlalchemy.engine.Engine [raw sql] ('Golden Lane', 0, 20, 0, 'dublin', 'GOLDEN LANE', 17, 53.340803, -6.267732, 'OPEN')
2023-02-22 23:35:25,995 INFO sqlalchemy.engine.Engine COMMIT
{'number': 45, 'contract_name': 'dublin', 'name': 'DEVERELL PLACE', 'address': 'Deverell Place', 'position': {'lat': 53.351464, 'lng': -6.255265},

2023-02-22 23:35:30,432 INFO sqlalchemy.engine.Engine [raw sql] ('Lime Street', 0, 40, 0, 'dublin', 'LIME STREET', 62, 53.346026, -6.243576, 'OPEN')
2023-02-22 23:35:30,555 INFO sqlalchemy.engine.Engine COMMIT
{'number': 5, 'contract_name': 'dublin', 'name': 'CHARLEMONT PLACE', 'address': 'Charlemont Street', 'position': {'lat': 53.330662, 'lng': -6.260177}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 0, 'available_bikes': 40, 'status': 'OPEN', 'last_update': 1677108603000}
2023-02-22 23:35:30,809 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 23:35:30,811 INFO sqlalchemy.engine.Engine [raw sql] ('Charlemont Street', 0, 40, 0, 'dublin', 'CHARLEMONT PLACE', 5, 53.330662, -6.260177, 'OPEN')
2023-02-22 23:35:30,931 INFO sqlalchemy.engine.Engine COMMIT
{'number': 97, 'contract_name': 'dublin', 'name': 'KILMAINHAM GAOL', 'address': 'Kilmainham Gaol', 'position': {'lat': 53.342113, 'lng': -

2023-02-22 23:35:35,326 INFO sqlalchemy.engine.Engine [raw sql] ("Sir Patrick's Dun", 0, 40, 0, 'dublin', "SIR PATRICK DUN'S", 58, 53.339218, -6.240642, 'OPEN')
2023-02-22 23:35:35,448 INFO sqlalchemy.engine.Engine COMMIT
{'number': 66, 'contract_name': 'dublin', 'name': 'NEW CENTRAL BANK', 'address': 'New Central Bank', 'position': {'lat': 53.347122, 'lng': -6.234749}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 27, 'available_bikes': 13, 'status': 'OPEN', 'last_update': 1677108589000}
2023-02-22 23:35:35,703 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 23:35:35,705 INFO sqlalchemy.engine.Engine [raw sql] ('New Central Bank', 0, 40, 0, 'dublin', 'NEW CENTRAL BANK', 66, 53.347122, -6.234749, 'OPEN')
2023-02-22 23:35:35,825 INFO sqlalchemy.engine.Engine COMMIT
{'number': 104, 'contract_name': 'dublin', 'name': 'GRANGEGORMAN LOWER (CENTRAL)', 'address': 'Grangegorman Lower (Central)',

...

2023-02-22 23:35:54,597 INFO sqlalchemy.engine.Engine [raw sql] ('St James Hospital (Luas)', 0, 40, 0, 'dublin', 'ST JAMES HOSPITAL (LUAS)', 80, 53.341359, -6.292951, 'OPEN')
2023-02-22 23:35:54,721 INFO sqlalchemy.engine.Engine COMMIT
{'number': 41, 'contract_name': 'dublin', 'name': 'HARCOURT TERRACE', 'address': 'Harcourt Terrace', 'position': {'lat': 53.332763, 'lng': -6.257942}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 8, 'available_bikes': 12, 'status': 'OPEN', 'last_update': 1677108790000}
2023-02-22 23:35:54,979 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 23:35:54,981 INFO sqlalchemy.engine.Engine [raw sql] ('Harcourt Terrace', 0, 20, 0, 'dublin', 'HARCOURT TERRACE', 41, 53.332763, -6.257942, 'OPEN')
2023-02-22 23:35:55,110 INFO sqlalchemy.engine.Engine COMMIT
{'number': 3, 'contract_name': 'dublin', 'name': 'BOLTON STREET', 'address': 'Bolton Street', 'position': {'lat'

2023-02-22 23:36:00,337 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:00,340 INFO sqlalchemy.engine.Engine [raw sql] (108, 9, 26, 1677108383000)
2023-02-22 23:36:00,462 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:00,715 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:00,717 INFO sqlalchemy.engine.Engine [raw sql] (20, 2, 28, 1677108624000)
2023-02-22 23:36:00,839 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:01,090 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:01,092 INFO sqlalchemy.engine.Engine [raw sql] (56, 17, 23, 1677108480000)
2023-02-22 23:36:01,216 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:01,468 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:01,470 INFO sqlalchemy.engine.Engine [raw sql] (6, 0, 19, 167

2023-02-22 23:36:11,757 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:12,013 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:12,015 INFO sqlalchemy.engine.Engine [raw sql] (9, 3, 21, 1677108853000)
2023-02-22 23:36:12,138 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:12,390 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:12,392 INFO sqlalchemy.engine.Engine [raw sql] (67, 9, 31, 1677108739000)
2023-02-22 23:36:12,513 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:12,766 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:12,768 INFO sqlalchemy.engine.Engine [raw sql] (116, 1, 29, 1677108564000)
2023-02-22 23:36:12,892 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:13,152 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:13

2023-02-22 23:36:23,375 INFO sqlalchemy.engine.Engine [raw sql] (71, 11, 29, 1677108679000)
2023-02-22 23:36:23,497 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:23,751 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:23,753 INFO sqlalchemy.engine.Engine [raw sql] (79, 0, 27, 1677108594000)
2023-02-22 23:36:23,876 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:24,128 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:24,131 INFO sqlalchemy.engine.Engine [raw sql] (69, 14, 26, 1677108500000)
2023-02-22 23:36:24,252 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:24,502 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:24,505 INFO sqlalchemy.engine.Engine [raw sql] (25, 0, 30, 1677108529000)
2023-02-22 23:36:24,628 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:24,880 INFO sqlalchemy.en

2023-02-22 23:36:35,197 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:35,200 INFO sqlalchemy.engine.Engine [raw sql] (109, 20, 9, 1677108514000)
2023-02-22 23:36:35,324 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:35,586 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:35,589 INFO sqlalchemy.engine.Engine [raw sql] (85, 13, 22, 1677108546000)
2023-02-22 23:36:35,709 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:35,963 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:35,966 INFO sqlalchemy.engine.Engine [raw sql] (107, 14, 26, 1677108717000)
2023-02-22 23:36:36,087 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:36:36,339 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:36:36,342 INFO sqlalchemy.engine.Engine [raw sql] (33, 12, 11,

In [73]:
NAME="Dublin"
STATIONS="https://api.jcdecaux.com/vls/v1/stations"
r = requests.get(STATIONS, params={"contract" : NAME, "apiKey": DubBike_API})

In [74]:
r.json()

[{'number': 42,
  'contract_name': 'dublin',
  'name': 'SMITHFIELD NORTH',
  'address': 'Smithfield North',
  'position': {'lat': 53.349562, 'lng': -6.278198},
  'banking': False,
  'bonus': False,
  'bike_stands': 30,
  'available_bike_stands': 9,
  'available_bikes': 21,
  'status': 'OPEN',
  'last_update': 1677108996000},
 {'number': 30,
  'contract_name': 'dublin',
  'name': 'PARNELL SQUARE NORTH',
  'address': 'Parnell Square North',
  'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526},
  'banking': False,
  'bonus': False,
  'bike_stands': 20,
  'available_bike_stands': 12,
  'available_bikes': 8,
  'status': 'OPEN',
  'last_update': 1677109150000},
 {'number': 54,
  'contract_name': 'dublin',
  'name': 'CLONMEL STREET',
  'address': 'Clonmel Street',
  'position': {'lat': 53.336021, 'lng': -6.26298},
  'banking': False,
  'bonus': False,
  'bike_stands': 33,
  'available_bike_stands': 28,
  'available_bikes': 5,
  'status': 'OPEN',
  'last_update': 1677108936000},
 

In [77]:
def availability_to_db(text):
    stations = json.loads(text)
    #print(availability)
    for station in stations:
        #print(station.get("name"))
        #print(station.get("number"))
        #print(station.get("available_bikes"))
        #print(station.get("available_bike_stands"))
        #print(station.get("last_update"))
        vals = (int(station.get("number")), int(station.get("available_bikes")), int(station.get("available_bike_stands")), int(station.get("last_update")))
        engine.execute("INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)", vals)
    return
        

In [78]:
# just testing out function works here
availability_to_db(r.text)

2023-02-22 23:45:24,678 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:45:24,683 INFO sqlalchemy.engine.Engine [raw sql] (42, 21, 9, 1677108996000)
2023-02-22 23:45:24,818 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:45:25,090 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:45:25,092 INFO sqlalchemy.engine.Engine [raw sql] (30, 8, 12, 1677109150000)
2023-02-22 23:45:25,224 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:45:25,474 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:45:25,476 INFO sqlalchemy.engine.Engine [raw sql] (54, 5, 28, 1677108936000)
2023-02-22 23:45:25,599 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:45:25,846 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:45:25,848 INFO sqlalchemy.engine.Engine [raw sql] (108, 9, 26, 167

2023-02-22 23:45:38,023 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:45:38,275 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:45:38,279 INFO sqlalchemy.engine.Engine [raw sql] (113, 21, 19, 1677108942000)
2023-02-22 23:45:38,399 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:45:38,654 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:45:38,655 INFO sqlalchemy.engine.Engine [raw sql] (91, 26, 4, 1677109178000)
2023-02-22 23:45:38,778 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:45:39,041 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:45:39,042 INFO sqlalchemy.engine.Engine [raw sql] (99, 2, 28, 1677108814000)
2023-02-22 23:45:39,166 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:45:39,418 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:45:

2023-02-22 23:45:49,678 INFO sqlalchemy.engine.Engine [raw sql] (96, 7, 23, 1677109160000)
2023-02-22 23:45:49,819 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:45:50,066 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:45:50,071 INFO sqlalchemy.engine.Engine [raw sql] (82, 15, 7, 1677108951000)
2023-02-22 23:45:50,195 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:45:50,446 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:45:50,448 INFO sqlalchemy.engine.Engine [raw sql] (76, 4, 34, 1677109254000)
2023-02-22 23:45:50,575 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:45:50,827 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:45:50,829 INFO sqlalchemy.engine.Engine [raw sql] (71, 11, 29, 1677109285000)
2023-02-22 23:45:50,953 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:45:51,204 INFO sqlalchemy.eng

2023-02-22 23:46:01,486 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:46:01,488 INFO sqlalchemy.engine.Engine [raw sql] (100, 25, 0, 1677108947000)
2023-02-22 23:46:01,611 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:46:01,860 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:46:01,863 INFO sqlalchemy.engine.Engine [raw sql] (24, 6, 14, 1677108845000)
2023-02-22 23:46:01,984 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:46:02,238 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:46:02,240 INFO sqlalchemy.engine.Engine [raw sql] (64, 24, 16, 1677108950000)
2023-02-22 23:46:02,364 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 23:46:02,628 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-22 23:46:02,629 INFO sqlalchemy.engine.Engine [raw sql] (109, 20, 9, 1